In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink

from basic_tools import *

In [2]:
import glob

# Load variables

In [3]:
plink_path=final_plink_path
plink_aa_path=final_plink_aa_path
#aa_path=final_aa_path

In [4]:
plink=PyPlink(plink_path)
fam=plink.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
bim=plink.get_bim()

In [5]:
plink_aa=PyPlink(plink_aa_path)
fam_aa=plink_aa.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
bim_aa=plink_aa.get_bim()

In [6]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')

In [7]:
phenotypes.columns

Index(['ID', 'age', 'sex', 'asthma', 'chronic_bronchitis', 'allergic_disease',
       'stomach_cancer', 'liver_cancer', 'colorectal_cancer', 'breast_cancer',
       ...
       'FVC_predicted', 'FEV_predicted', 'FEV_over_FVC_predicted',
       'MVV_predicted', 'bone_density_distal_sos', 'bone_density_midshaft_sos',
       'protein_in_blood', 'glucose_in_blood', 'cohort', 'bmi'],
      dtype='object', length=138)

In [8]:
phenotype_list=sorted([path.split('/')[-1] for path in glob.glob(data_out_assoc_path+'*')])

In [9]:
pd.Index(phenotypes.columns[~phenotypes.columns.str.contains('x_ray')]).difference(phenotype_list)

Index(['FEV_over_FVC_predicted', 'FEV_predicted', 'FVC_predicted', 'ID',
       'MVV_predicted', 'acute_liver_disease', 'age',
       'bone_density_distal_sos', 'bone_density_midshaft_sos', 'diabetes',
       'diastolic_blood_pressure', 'duodenal_ulcer', 'fatty_liver', 'fracture',
       'hypertension', 'hysterectomy', 'insomnia', 'liver_cancer',
       'menarche_onset_age', 'menopause_age', 'menopause_cycle', 'neurosis',
       'osteoporosis', 'prostate_cancer', 'protein_in_blood', 'thyroid_cancer',
       'total_cholesterol', 'transient_ischemic_attacks', 'triglyceride',
       'tsh', 'tuberculosis', 'weight'],
      dtype='object')

In [10]:
gene_bed_path='data/known_genes_chr6.hg18.txt'

In [11]:
gene_bed=pd.read_csv(gene_bed_path,sep='\t')
gene_bed=gene_bed[(gene_bed['txStart']>=bim_aa.pos.min())&(gene_bed['txEnd']<=bim_aa.pos.max())]

# HLA_gene_assign

In [12]:
gene_assign_HLA=bim_aa[['pos']]
HLA_names=np.unique([i[0].split('_')[1] for i in bim_aa[bim_aa.index.str.contains('HLA_')].index.str.split('*')])

for HLA_name in HLA_names:
    gene_assign_HLA[HLA_name]=0
    gene_select=gene_assign_HLA[gene_assign_HLA.index.str.contains('HLA_'+HLA_name)|gene_assign_HLA.index.str.contains('SNPS_'+HLA_name)|gene_assign_HLA.index.str.contains('AA_'+HLA_name)]#print(gene_select.sort_values('pos').iloc[0],gene_select.sort_values('pos').iloc[-1])
    gene_assign_HLA[HLA_name][(gene_assign_HLA['pos']>=gene_select['pos'].min())&(gene_assign_HLA['pos']<=gene_select['pos'].max())]=1
#gene_assign    

/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://p

# Non HLA gene assign

In [13]:
gene_assign_nonHLA=bim_aa[['pos']]
for idx,row in gene_bed.iterrows():
    gene_assign_nonHLA[row['name2']]=0
    
for idx,row in gene_bed.iterrows():
    exon_range_list=list(zip(row['exonStarts'].strip(',').split(','),row['exonEnds'].strip(',').split(',')))
    for start,end in exon_range_list:
        start=int(start);end=int(end)
        gene_assign_nonHLA_bool=((gene_assign_nonHLA['pos']>=start) & (gene_assign_nonHLA['pos']<=end))
        gene_assign_nonHLA[row['name2']][gene_assign_nonHLA_bool]=1    

/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Gene assignment

In [19]:
loci_info_list=[]
for phenotype_name in phenotype_list:
    #phenotype_name='ALP'
    print(phenotype_name)
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
    num_independent=len(glob.glob(data_out_assoc_phenotype_path+'step_*.aa'))
    phenotype_data=pd.read_csv(data_out_assoc_phenotype_path+'phenotype.phe',sep='\t',header=None,names=['FID','IID','pheno'])
    phenotype_type= 'continuous' if len(phenotype_data['pheno'].unique())>3 else 'binary'
    
    for idx in range(1,num_independent+1):
        omnibus_assoc_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.assoc.munged'.format(idx),sep='\t')
        plink_assoc_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.assoc.{}.munged'.format(idx,'logistic' if phenotype_type=='binary' else 'linear')).iloc[:,1:]
        
        plink_cond_f=open(data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(5),'r')
        plink_cond=[cond.strip() for cond in plink_cond_f.readlines()]
        plink_cond_f.close()
        plink_covar=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(1),sep='\t')
            
        omnibus_top=omnibus_assoc_result.sort_values('P').iloc[0]
        plink_top=plink_assoc_result.sort_values('P').iloc[0]
            
        assoc_type='plink' if plink_top['P']<omnibus_top['P'] else 'omnibus'
        
        variant=plink_top['SNP'] if assoc_type=='plink' else  omnibus_top['Variant']
        pos=bim_aa.loc[plink_top['SNP']]['pos'] if assoc_type=='plink' else  omnibus_top['Variant']
        p_val=plink_top['P'] if assoc_type=='plink' else omnibus_top['P']
        
        if assoc_type=='omnibus':
            gene_name='HLA_'+variant.split('_')[1]
        elif assoc_type=='plink':
            if variant[:3]=='HLA':
                gene_name='HLA_'+variant[4:].split('*')[0]
            else:
                # if the variant is itself in HLA transcription region
                if gene_assign_HLA.iloc[:,1:].loc[variant].sum()==1:
                    gene_name='HLA_'+gene_assign_HLA.iloc[:,1:].loc[variant].sort_values(ascending=False).index[0]
                elif gene_assign_HLA.iloc[:,1:].loc[variant].sum()>1:
                    raise
                # if the variant is itself in non-HLA gene coding region
                elif gene_assign_nonHLA.iloc[:,1:].loc[variant].sum()==1:
                    gene_name='HLA_'+gene_assign_nonHLA.iloc[:,1:].loc[variant].sort_values(ascending=False).index[0]
                elif gene_assign_nonHLA.iloc[:,1:].loc[variant].sum()>1:
                    raise
                else:
                    gene_assign_HLA_select=gene_assign_HLA[r2_df[0]>0.5].iloc[:,1:]
                    gene_assign_nonHLA_select=gene_assign_nonHLA[r2_df[0]>0.7].iloc[:,1:]
                    # if the variant is in strong LD(>0.7) with a variant in HLA transcription region
                    if gene_assign_HLA_select.sum(axis=0).sum()==1: 
                        gene_name=gene_assign_HLA_select.sum(axis=0).sort_values(ascending=False).index[0]
                    elif gene_assign_HLA_select.sum(axis=0).sum()>1:
                        gene_assign_HLA_select_select_index=gene_assign_HLA_select.sum(axis=1).sort_values(ascending=False).index[gene_assign_HLA_select.sum(axis=1).sort_values(ascending=False)>0]
                        gene_assign_HLA_select_select_index_sorted=r2_df.loc[gene_assign_HLA_select_select_index].sort_values(0,ascending=False).index
                        if gene_assign_HLA_select.loc[gene_assign_HLA_select_select_index_sorted[0]].sum()==1:
                            gene_name=gene_assign_HLA_select.loc[gene_assign_HLA_select_select_index_sorted[0]].sort_values(ascending=False).index[0]
                        else:
                            raise
                    # if the variant is in moderate LD(>0.5) with a variant in non-HLA gene coding region
                    elif gene_assign_nonHLA_select.sum(axis=0).sum()==1:
                        gene_name=gene_assign_nonHLA_select.sum(axis=0).sort_values(ascending=False).index[0]
                    elif gene_assign_nonHLA_select.sum(axis=0).sum()>1:
                        gene_assign_nonHLA_select_select_index=gene_assign_nonHLA_select.sum(axis=1).sort_values(ascending=False).index[gene_assign_nonHLA_select.sum(axis=1).sort_values(ascending=False)>0]
                        gene_assign_nonHLA_select_select_index_sorted=r2_df.loc[gene_assign_nonHLA_select_select_index].sort_values(0,ascending=False).index
                        if gene_assign_nonHLA_select.loc[gene_assign_nonHLA_select_select_index_sorted[0]].sum()==1:
                            gene_name=gene_assign_nonHLA_select.loc[gene_assign_nonHLA_select_select_index_sorted[0]].sort_values(ascending=False).index[0]
                        else:
                            raise
                    #in other cases, assign to the nearest gene
                    else:
                        gene_bed_copy=gene_bed.copy()
                        gene_bed_copy['txStartDist']=(gene_bed_copy['txStart']-pos).abs()
                        gene_bed_copy['txEndDist']=(gene_bed_copy['txEnd']-pos).abs()
                        gene_bed_copy['txDist']=gene_bed_copy[['txStartDist','txEndDist']].min(axis=1)
                        gene_name=gene_bed_copy.sort_values('txDist').iloc[0]['name2']     
        
        
        loci_info={'phenotype_name':phenotype_name,
                   
                   'variant': variant,
                   'pos':pos,
                   'P':p_val,
                   
                   'Gene':gene_name,
                    'total':(phenotype_data['pheno']!=-9).sum(),
                   'case':(phenotype_data['pheno']==2).sum() if phenotype_type=='binary' else (phenotype_data['pheno']!=-9).sum(),
                   'control':(phenotype_data['pheno']==1).sum() if phenotype_type=='binary' else 0,
                   'A1/A2':((plink_top['A1'],(bim.loc[plink_top['SNP']]['a2'] if plink_top['A1']!=bim.loc[plink_top['SNP']]['a2'] else bim.loc[plink_top['SNP']]['a1']))) if assoc_type=='plink' else ('-','-'),
                   'Z(beta)': '{}({})'.format(plink_top['STAT'],plink_top['BETA']) if assoc_type=='plink' else '-',
                   'chisq(df)':'{}({})({})'.format(omnibus_top['chisq'],omnibus_top['deltaDF'],','.join(omnibus_top['Residues'])) if assoc_type=='omnibus' else '-',
                   'conditional_covariate':plink_cond+plink_covar.columns.tolist()
                  }
        
        loci_info_list.append(loci_info)
        if idx==num_independent:
            p_val>5e-8:
                raise

ALP
DRB1    0
DQB1    0
DQA1    0
DPB1    0
DPA1    0
C       0
B       0
A       0
dtype: int64
TRIM26    2
RNF39     1
TRIM40    1
TRIM15    0
TRIM10    0
         ..
RDBP      0
CFB       0
C2        0
ZBTB12    0
OR2B6     0
Length: 229, dtype: int64


RuntimeError: No active exception to reraise